<a href="https://colab.research.google.com/github/shawk17/563-shawk17/blob/main/Hoomd_Notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 2
## Hoomd

### Install

3 Ways to install, use Conda (uses pip), pip

In [1]:
# Install
!pip install -q condacolab
# Creates a repository
!git clone --single-branch --branch cecam https://github.com/cmelab/hoomd-organics
# This will crash the session, restart collab and changes seesion
import condacolab
condacolab.install()

Cloning into 'hoomd-organics'...
remote: Enumerating objects: 3573, done.
remote: Counting objects: 100% (829/829), done.
remote: Compressing objects: 100% (183/183), done.
remote: Total 3573 (delta 674), reused 646 (delta 646), pack-reused 2744 (from 1)
Receiving objects: 100% (3573/3573), 1.53 MiB | 10.71 MiB/s, done.
Resolving deltas: 100% (2461/2461), done.
⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:16
🔁 Restarting kernel...


In [1]:
#!pip install --upgrade ipykernel #breaks things?
import os
os.chdir("hoomd-organics")
# For packaging .yml to organize depencies
!mamba env update -n base -f environment-cpu.yml
!python -m pip install -e .
import warnings
warnings.filterwarnings('ignore')

Channels:
 - conda-forge
Platform: linux-64
Solving environment: - \ | / - \ | / - \ done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.2
    latest version: 25.1.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



cudatoolkit-11.8.0   | 682.5 MB  | :   0% 0/1 [00:00<?, ?it/s]
gcc_impl_linux-64-14 | 70.1 MB   | :   0% 0/1 [00:00<?, ?it/s]

pillow-11.1.0        | 40.1 MB   | :   0% 0/1 [00:00<?, ?it/s]


libllvm14-14.0.6     | 30.0 MB   | :   0% 0/1 [00:00<?, ?it/s]



libflint-3.1.3.1     | 20.2 MB   | :   0% 0/1 [00:00<?, ?it/s]




rdkit-2024.09.6      | 19.0 MB   | :   0% 0/1 [00:00<?, ?it/s]





parmed-4.3.0         | 18.5 MB   | :   0% 0/1 [00:00<?, ?it/s]






scipy-1.15.2         | 16.4 MB   | :   0% 0/1 [00:00<?, ?it/s]







libclang-cpp14-14.0. | 15.8 MB   | :   0% 0/1 [00:00<?, ?it/s]








pandas-2.2.3         | 15.0 MB   | :   0% 0/1 [00:00<?, ?it/s]









sysroot_linux-64-2.1 | 

## Using Hoomd
* Initialize LJ system
* run NVE simulation

In [4]:
import hoomd
import matplotlib.pyplot as plt
import numpy as np

cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=1)

print(sim.state)
print(sim.operations.integrator)
print(sim.operations.updaters[:])
print(sim.operations.writers[:])

None
None
[]
[]


In [11]:
# This is not intended as a full tutorial on fresnel - see the fresnel user
# documentation (https://fresnel.readthedocs.io/) if you would like to learn more.

import warnings

import fresnel
import IPython
import packaging.version

device = fresnel.Device()
tracer = fresnel.tracer.Path(device=device, w=300, h=300)

FRESNEL_MIN_VERSION = packaging.version.parse("0.13.0")
FRESNEL_MAX_VERSION = packaging.version.parse("0.14.0")


def render(position, orientation, L):
    if (
        "version" not in dir(fresnel)
        or packaging.version.parse(fresnel.version.version) < FRESNEL_MIN_VERSION
        or packaging.version.parse(fresnel.version.version) >= FRESNEL_MAX_VERSION
    ):
        warnings.warn(
            f"Unsupported fresnel version {fresnel.version.version} - expect errors."
        )
    vertices = [
        (-0.5, 0, 0),
        (0.5, 0, 0),
        (0, -0.5, 0),
        (0, 0.5, 0),
        (0, 0, -0.5),
        (0, 0, 0.5),
    ]
    poly_info = fresnel.util.convex_polyhedron_from_vertices(vertices)

    scene = fresnel.Scene(device)
    geometry = fresnel.geometry.ConvexPolyhedron(scene, poly_info, N=len(position))
    geometry.material = fresnel.material.Material(
        color=fresnel.color.linear([0.01, 0.74, 0.26]), roughness=0.5
    )
    geometry.position[:] = position[:]
    geometry.orientation[:] = orientation[:]
    geometry.outline_width = 0.01
    fresnel.geometry.Box(scene, [L, L, L, 0, 0, 0], box_radius=0.02)

    scene.lights = [
        fresnel.light.Light(direction=(0, 0, 1), color=(0.8, 0.8, 0.8), theta=math.pi),
        fresnel.light.Light(
            direction=(1, 1, 1), color=(1.1, 1.1, 1.1), theta=math.pi / 3
        ),
    ]
    scene.camera = fresnel.camera.Orthographic(
        position=(L * 2, L, L * 2), look_at=(0, 0, 0), up=(0, 1, 0), height=L * 1.4 + 1
    )
    scene.background_color = (1, 1, 1)
    scene.background_alpha = 1
    samples = 2000
    if "CI" in os.environ:
        samples = 100
    return IPython.display.Image(tracer.sample(scene, samples=samples)._repr_png_())

In [12]:
mc = hoomd.hpmc.integrate.ConvexPolyhedron()
mc.shape["square"] = dict(
    vertices=[
        [-0.5, -0.5, -0.5],
        [-0.5, -0.5, 0.5],
        [-0.5, 0.5, -0.5],
        [-0.5, 0.5, 0.5],
        [0.5, -0.5, -0.5],
        [0.5, -0.5, 0.5],
        [0.5, 0.5, -0.5],
        [0.5, 0.5, 0.5],
    ]
)
mc.nselect = 2 # Trial moves
mc.d["square"] = 0.15 # Distance
mc.a["square"] = 0.2  # Rotations
sim.operations.integrator = mc


In [14]:
# Defining the system state
import itertools
import math
import numpy as np

m = 4
N_particles = m**3
spacing = 1.2
K = math.ceil(N_particles**(1/3))
L = K*spacing
x = np.linspace(-L/2, L/2, K, endpoint=False)
position = list(itertools.product(x, repeat=3))
print(position)
position = position[:N_particles]
print(position)
orientation = [(1,0,0,0)] * N_particles
#render(position, orientation, L)

[(-2.4, -2.4, -2.4), (-2.4, -2.4, -1.2), (-2.4, -2.4, 0.0), (-2.4, -2.4, 1.1999999999999997), (-2.4, -1.2, -2.4), (-2.4, -1.2, -1.2), (-2.4, -1.2, 0.0), (-2.4, -1.2, 1.1999999999999997), (-2.4, 0.0, -2.4), (-2.4, 0.0, -1.2), (-2.4, 0.0, 0.0), (-2.4, 0.0, 1.1999999999999997), (-2.4, 1.1999999999999997, -2.4), (-2.4, 1.1999999999999997, -1.2), (-2.4, 1.1999999999999997, 0.0), (-2.4, 1.1999999999999997, 1.1999999999999997), (-1.2, -2.4, -2.4), (-1.2, -2.4, -1.2), (-1.2, -2.4, 0.0), (-1.2, -2.4, 1.1999999999999997), (-1.2, -1.2, -2.4), (-1.2, -1.2, -1.2), (-1.2, -1.2, 0.0), (-1.2, -1.2, 1.1999999999999997), (-1.2, 0.0, -2.4), (-1.2, 0.0, -1.2), (-1.2, 0.0, 0.0), (-1.2, 0.0, 1.1999999999999997), (-1.2, 1.1999999999999997, -2.4), (-1.2, 1.1999999999999997, -1.2), (-1.2, 1.1999999999999997, 0.0), (-1.2, 1.1999999999999997, 1.1999999999999997), (0.0, -2.4, -2.4), (0.0, -2.4, -1.2), (0.0, -2.4, 0.0), (0.0, -2.4, 1.1999999999999997), (0.0, -1.2, -2.4), (0.0, -1.2, -1.2), (0.0, -1.2, 0.0), (0.0, 

## Class Code

In [ ]:
import hoomd
import numpy as np

device = hoomd.device.CPU()
sim = hoomd.Simulation(device=device, seed=np.random.randint(0, 2**32 - 1))
kT = 1


In [15]:
# An a analysis pckage
import freud

starting_density = 0.5
a = 1/(starting_density**(1/3.0))
num_replicas = 10
N_particles = num_replicas**3

grid_particles = freud.data.UnitCell([a,a,a,0,0,0], [[0,0,0]]).generate_system(num_replicas)
box_length = grid_particles[0].Lx

In [ ]:
import gsd.hoomd
frame =  gsd.hoomd.Frame()
frame.particles.N = N_particles
frame.particles.position = grid_particles
frame.configuration.box = [box_length, box_length, box_length, 0, 0, 0]

frame.particles.typeid = [0]*N_particles
frame.particles.types = ['A']

with gsd.hoomd.open(name='initial_state.gsd', mode='w') as f:
    f.append(frame)

In [ ]:
sim.create_state_from_gsd(filename='initial_state.gsd')